In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

aurora_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
nugget_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'
query_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv'

data_file = csvigo.load({path = aurora_fn, mode = "large"})
nugget_file = csvigo.load({path = nugget_fn, mode = "large"})
query_file =  csvigo.load({path = query_fn, mode = "large"})

rK = 100
nbatches = 50
nepochs = 100
print_every = 10
embed_dim = 10
learning_rate = 0.1

cuts = 4.                  --- This is the number of cuts we want
epsilon = 1.
base_explore_rate = 0.1
delta = 1./(nepochs/cuts) --- Only using epsilon greedy strategy for (nepochs/cuts)% of the epochs

cuda = true
torch.manualSeed(420)

...Utils file loaded	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv	
<csv>	parsing done	


In [2]:
function build_network(vocab_size, embed_dim)
    model = nn.Sequential()
    :add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
    :add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
    :add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
    :add(nn.SelectTable(-1)) -- selects last state of the LSTM
    :add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification
    :add(nn.ReLU())
   return model
end
function build_model(vocab_size, embed_dim, outputSize)
    lstm1 = build_network(vocab_size, embed_dim)
    lstm2 = build_network(vocab_size, embed_dim)
    lstm3 = build_network(vocab_size, embed_dim)

    mlp1 = nn.Sequential()
    mlp1:add(nn.Linear(1, embed_dim))
    mlp1:add(nn.ReLU())

    ParallelModel = nn.ParallelTable()
    ParallelModel:add(lstm1)
    ParallelModel:add(lstm2)
    ParallelModel:add(lstm3)
    ParallelModel:add(mlp1)

    FinalMLP = nn.Sequential()
    FinalMLP:add(ParallelModel)
    FinalMLP:add(nn.JoinTable(2))
    FinalMLP:add( nn.Linear(embed_dim * 4, outputSize) )
    return FinalMLP
end

In [3]:
vocab_sized = getVocabSize(data_file)                       --- getting length of dictionary
vocab_sizeq = getVocabSize(query_file)                      --- getting length of dictionary
vocab_size = math.max(vocab_sized, vocab_sizeq)

queries = grabNsamples(query_file, #query_file-1, nil)      --- Extracting all queries
nuggets = grabNsamples(nugget_file, #nugget_file-1, nil)    --- Extracting all samples
maxlend = getMaxseq(data_file)                             --- Extracting maximum sequence length
maxlenq = getMaxseq(query_file)                            --- Extracting maximum sequence length
maxlen = math.max(maxlenq, maxlend)

batchLSTM = build_model(vocab_size, embed_dim, 1)
crit = nn.MSECriterion()

In [ ]:
dofile("model_utils.lua")

In [4]:
out = iterateModel( nbatches, nepochs, queries[3], data_file, batchLSTM, crit, epsilon, delta, 
                    maxlen, base_explore_rate, print_every, nuggets, learning_rate, rK)

utils.lua:250: bad argument #1 to 'pairs' (table expected, got userdata)
stack traceback:
	[C]: in function 'pairs'
	utils.lua:250: in function 'geti_n'
	model_utils.lua:71: in function 'iterateModel'
	[string "out = iterateModel( nbatches, nepochs, querie..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0102933b90: 

In [13]:
rscores, pscores, fscores = {}, {}, {}

In [14]:
x = data_file
K = rK

In [22]:
preds_list = torch.totable(torch.round(torch.rand(#x)))
yrouge = torch.totable(torch.randn(#x))
summary_list = populateOnes(#x, K)
action_list = torch.totable(torch.round(torch.randn(#x)))

In [16]:
qs = queries[3]

In [17]:
nstart = 2
nend = 20

In [19]:
x_ss  = geti_n(x, nstart, nend)
xout  = grabNsamples(x_ss, 1, #x_ss)
xs  = padZeros(xout, maxlen)
qs2 = padZeros({qs}, 5)
qrep = repeatQuery(qs2[1], #xs)

In [28]:
preds = geti_n(preds_list, nstart, nend)

In [33]:
sumry_ss = buildPredSummary(preds, xs)

In [37]:
sumry_ss2 = padZeros(sumry_ss, maxlen)

In [49]:
yr_ss = geti_n(yrouge, nstart, nend)
as_ss = geti_n(action_list, nstart, nend)
sentences = torch.LongTensor(xs):t()
summary = torch.LongTensor(sumry_ss2):t()
query = torch.LongTensor(qrep):t()
actions = torch.Tensor(as_ss):resize(#xs, 1)
labels = torch.Tensor(yr_ss)

In [39]:
model = batchLSTM

In [50]:
myPreds = model:forward({sentences, summary, query, actions})

In [ ]:
loss = loss + crit:forward(myPreds, labels)
grads = crit:backward(myPreds, labels)
model:backward({sentences, summary, query, actions}, grads)
model:updateParameters(learning_rate)        -- Update parameters after each minibatch
model:zeroGradParameters()

In [ ]:
preds = policy(myPreds, epsilon, #xs)
--- Concatenating predictions into a summary
predsummary = buildPredSummary(preds, xs)
--- Initializing rouge metrics at time {t-1} and save scores
for i=1, #predsummary do
    --- Calculating rouge scores; Call get_i_n() to cumulatively compute rouge
    rscores[i] = rougeRecall(geti_n(predsummary, 1, i), nuggets, K) - r_t1
    pscores[i] = rougePrecision(geti_n(predsummary, 1, i), nuggets, K) - p_t1
    fscores[i] = rougeF1(geti_n(predsummary, 1, i), nuggets, K) - f_t1
    r_t1, p_t1, f_t1 = rscores[i], pscores[i], fscores[i]
end
--- Updating change in rouge
yrouge = updateTable(yrouge, fscores, nstart)
preds_list = updateTable(preds_list, preds, nstart)
--- Calculating last one to see actual last rouge, without delta
rscore = rougeRecall(predsummary, nuggets, K)
pscore = rougePrecision(predsummary, nuggets, K)
fscore = rougeF1(predsummary, nuggets, K)
